# Queens and Star Battle

https://www.reddit.com/r/puzzles/comments/1nlp2rq/queens/

## Introduction: Problem Definition and Computational Complexity

This is a constraint satisfaction puzzle, fairly easily reducible to SAT.

### 0-1 Integer Linear Constraint Satisfiability Formulation

Specifically, I've set up the constraints in z3 using pseudo-boolean constraints, making it a 0-1 integer linear programming problem (BILP) in which only the constraints need to be satisfied (no optimization), and therefore one of Karp's 21 NP-Complete problems, and therefore reducible to SAT.

Here are the constraints written in familiar mathematical notation:

$$ x_{i,j} \in \mathbb{Z}_2 $$

All rows must have exactly $L$ queens/stars.
$$ \sum_{j} x_{i,j} = L \qquad \forall i $$

All columns must have exactly $L$ queens/stars.
$$ \sum_{i} x_{i,j} = L \qquad \forall j $$

All shapes must have exactly $L$ queens/stars.
$$ \sum_{x_{i,j} \in S_{k}} x_{i,j} = L \qquad \forall k $$

All queens/stars cannot neighbor eachother.
$$ x_{i,j} + x_{i',j'} \leq 1 \qquad \forall i,j,i',j' \colon (|i - i'| \leq 1) \land (|j - j'| \leq 1) \land \lnot (i=i' \land j=j')$$

We must have $\left| S \right| = n$ (each queen, or $L$ stars, has their own color shape, but not necessarily $\left| S_k \right| = n$. This gives us a total of $3n + 4n(n-1) = 4n^{2}-n$ linear contraints.

Where:
- $n$ is the number of rows and columns, i.e. we have an $n \times n$ grid.
- $x_{i,j}$ is the boolean which is true if cell $(i,j)$ has a queen/star.
- $L$ is the number of queens/stars in each row, column, and shape. For classic Queens $L=1$.
- $S$ is the set of all colored shapes. Each colored shape, $S_k$, being a set of cells $x_{i,j}$.


### Queens Reduction to SAT

For the $L=1$ case, classic Queens, the polynomial time reduction to SAT for this problem is particularly easy: Each cell is a boolean and its material implication is all others in its row, column, color and adjacency are False. The largest one of these clauses could be is a puzzle in which every color has only 1 cell (where its queen is) except one which has the remainder of the board: that color will have $n^{2} - \left( n - 1 \right)$ cells. Thus the total SAT problem reduction is $O(n^4)$. This actually would be a pretty easy alternative formulation in z3.


### References

- https://en.wikipedia.org/wiki/Boolean_satisfiability_problem
- https://en.wikipedia.org/wiki/Pseudo-Boolean_function
- https://en.wikipedia.org/wiki/Integer_programming
- https://en.wikipedia.org/wiki/Karp%27s_21_NP-complete_problems
- https://en.wikipedia.org/wiki/Polynomial-time_reduction
- https://en.wikipedia.org/wiki/Material_conditional

## Tools

### Transcription

In [ ]:
from PIL import Image, ImageDraw, ImageFilter

In [ ]:
import numpy as np

In [ ]:
def transcribe_grid(img, n, verbose=False):
    dx, dy = np.array(img.size) // n
    if verbose:
        print(f"{n} columns, spaced {dx}")
        print(f"{n} rows, spaced {dy}")
        timg = Image.new(img.mode, (n,n))
        
    # blur so as to remove texture and get a consistent mode
    img = img.convert('RGB')
    blured_img = img.filter(ImageFilter.BoxBlur(radius=dx//4))

    cmap = dict()
    grid = np.zeros((n,n),dtype=int)
    for i in range(n):
        for j in range(n):
            cimg = img.crop((j * dx, i * dy, (j+1) * dx, (i+1) * dy))
            # mode color
            arr = np.array(cimg)
            pixels = arr.reshape(-1, arr.shape[-1])
            uniq, counts = np.unique(pixels, axis=0, return_counts=True)
            mode_color = uniq[counts.argmax()]
            mode_color_hex = '#'+''.join(f"{c:02x}" for c in mode_color)
            
            if mode_color_hex not in cmap:
                cmap[mode_color_hex] = len(cmap)
            
            grid[i,j] = cmap[mode_color_hex]
            
            if verbose:
                print(f"Processed row {i} / {n}, column {j} / {n}: {mode_color_hex} = {cmap[mode_color_hex]}")
                timg.putpixel((j, i), tuple(mode_color))
    if verbose:
        return grid, timg
    return grid

### Solver Setup

In [ ]:
import z3

In [ ]:
def create_grid_vars(n):
    grid_vars = []
    for i in range(n):
        grid_vars.append([])
        for j in range(n):
            grid_vars[-1].append(z3.Bool(f"x_{i}_{j}"))
    return grid_vars

In [ ]:
def setup_solver(grid, grid_vars, l=1):
    n = len(grid_vars)
    solver = z3.Solver()
    
    # l per row
    for i in range(n):
        vs = [grid_vars[i][j] for j in range(n)]
        solver.add(z3.AtMost(*vs, l))
        solver.add(z3.AtLeast(*vs, l))

    # l per col
    for j in range(n):
        vs = [grid_vars[i][j] for i in range(n)]
        solver.add(z3.AtMost(*vs, l))
        solver.add(z3.AtLeast(*vs, l))
    
    # no adjacent
    #for i in range(n):
    #    for j in range(n):
    #        for di in [-1,0,1]:
    #            for dj in [-1,0,1]:
    #                if (di == 0 and dj == 0) or not (0 <= i+di < n) or not (0 <= j+dj < n):
    #                    continue
    #                solver.add(z3.AtMost(grid_vars[i][j], grid_vars[i+di][j+dj], 1))

    # left-right
    for i in range(n):
        for j in range(n-1):
            solver.add(z3.AtMost(grid_vars[i][j], grid_vars[i][j+1], 1))
    # up-down
    for i in range(n-1):
        for j in range(n):
            solver.add(z3.AtMost(grid_vars[i][j], grid_vars[i+1][j], 1))
    # diag-down right
    for i in range(n-1):
        for j in range(n-1):
            solver.add(z3.AtMost(grid_vars[i][j], grid_vars[i+1][j+1], 1))
    # diag-down left
    for i in range(n-1):
        for j in range(n-1):
            solver.add(z3.AtMost(grid_vars[i][j+1], grid_vars[i+1][j], 1))
    
    # l per color
    color_sets = [[] for i in range(n)]
    for i in range(n):
        for j in range(n):
            color_sets[grid[i,j]].append(grid_vars[i][j])
    for i in range(n):
        solver.add(z3.AtMost(*color_sets[i], l))
        solver.add(z3.AtLeast(*color_sets[i], l))
    
    return solver

In [ ]:
def complete_solve(grid, l = 1):
    n = grid.shape[0]
    
    grid_vars = create_grid_vars(n)
    solver = setup_solver(grid, grid_vars, l = l)
    #%%time
    if solver.check() != z3.sat:
        print("Unsolvable")
        return
    
    model = solver.model()
    solution = np.array([[bool(model.evaluate(grid_vars[i][j])) for j in range(n)] for i in range(n)])
    
    return solution

### Visualization

In [ ]:
def draw_solution(img, solution):
    n = solution.shape[0]
    dx, dy = np.array(img.size) // n
    ddx,ddy = dx//4, dy//4
    
    draw = ImageDraw.Draw(img)

    for i in range(n):
        for j in range(n):
            if solution[i,j]:
                draw.ellipse((j * dx + ddx, i * dy + ddy, (j+1) * dx - ddx, (i+1) * dy - ddy), fill='black')
                
    return img

## Example 1: u/low_on_vitamin_D's from Reddit

Original post [here](https://www.reddit.com/r/puzzles/comments/1nlp2rq/queens/)

### Fetch the Data

In [ ]:
import requests
import io

In [ ]:
url = "https://preview.redd.it/ex1q4jpb29qf1.jpeg?width=1080&crop=smart&auto=webp&s=3de9aeaaa9379ea49652314eb8b991ddc6868b2e"
puzzle_img = Image.open(io.BytesIO(requests.get(url).content))
puzzle_img.resize((puzzle_img.size[0]//4, puzzle_img.size[1]//4))

### Apply the Transcription

In [ ]:
cropped_img = puzzle_img.crop(np.array(puzzle_img.size * 2) * np.array([0.05, 0.21, 0.95, 0.63]))
cropped_img.resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

In [ ]:
n = 10

In [ ]:
grid, timg = transcribe_grid(cropped_img, n, verbose=True)
grid

In [ ]:
timg.resize((n*32,n*32), Image.NEAREST)

### Apply Corrections

In [ ]:
# None

### Apply to Solver

In [ ]:
solution = complete_solve(grid)
solution

In [ ]:
draw_solution(cropped_img.copy(), solution).resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

## Example 2: u/turbo488's from Reddit

Original post [here](https://www.reddit.com/r/puzzles/comments/1nh7wyx/queens_puzzle/)

### Fetch the Data

In [ ]:
import requests
import io

In [ ]:
url = "https://preview.redd.it/queens-puzzle-v0-sujcp4kr68pf1.jpeg?width=640&crop=smart&auto=webp&s=108d1d51633d06c877b67bf90920253f1d994687"
puzzle_img = Image.open(io.BytesIO(requests.get(url).content))
puzzle_img.resize((puzzle_img.size[0]//4, puzzle_img.size[1]//4))

### Apply the Transcription

In [ ]:
cropped_img = puzzle_img.crop(np.array(puzzle_img.size * 2) * np.array([0.05, 0.19, 0.95, 0.62]))
cropped_img.resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

In [ ]:
n = 8

In [ ]:
grid, timg = transcribe_grid(cropped_img, n, verbose=True)
grid

In [ ]:
timg.resize((n*32,n*32), Image.NEAREST)

### Apply Corrections

In [ ]:
# None

### Apply to Solver

In [ ]:
solution = complete_solve(grid)
solution

In [ ]:
draw_solution(cropped_img.copy(), solution).resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

## Example 3: u/Flamtart0's from Reddit

Original post [here](https://www.reddit.com/r/puzzles/comments/1n4ksq1/how_to_even_start/)

### Fetch the Data

In [ ]:
import requests
import io

In [ ]:
url = "https://preview.redd.it/how-to-even-start-v0-n58x4758z9mf1.png?width=640&crop=smart&auto=webp&s=e5a560eb3d8018d368a1312aa332f24371f0310d"
puzzle_img = Image.open(io.BytesIO(requests.get(url).content))
puzzle_img.resize((puzzle_img.size[0]//2, puzzle_img.size[1]//2))

### Apply the Transcription

In [ ]:
cropped_img = puzzle_img.crop(np.array(puzzle_img.size * 2) * np.array([0.15, 0.11, 0.93, 0.95]))
cropped_img.resize((cropped_img.size[0]//2, cropped_img.size[1]//2))

In [ ]:
n = 10

In [ ]:
grid, timg = transcribe_grid(cropped_img, n, verbose=True)
grid

In [ ]:
timg.resize((n*32,n*32), Image.NEAREST)

### Apply Corrections

In [ ]:
# None

### Apply to Solver

In [ ]:
solution = complete_solve(grid, l = 2)
solution

In [ ]:
draw_solution(cropped_img.copy(), solution).resize((cropped_img.size[0]//2, cropped_img.size[1]//2))

## Example 4: u/LifeInTheAbyss's from Reddit

Original post [here](https://www.reddit.com/r/puzzles/comments/1nh03zz/queens_lvl_40_how_to_proceed_next/)

### Fetch the Data

In [ ]:
import requests
import io

In [ ]:
url = "https://preview.redd.it/queens-lvl-40-how-to-proceed-next-v0-imlk5b8bj6pf1.png?width=1080&crop=smart&auto=webp&s=3f4ea59d16b18a03c7c7f215c5d25869d5b59ec9"
puzzle_img = Image.open(io.BytesIO(requests.get(url).content))
puzzle_img.resize((puzzle_img.size[0]//4, puzzle_img.size[1]//4))

### Apply the Transcription

In [ ]:
cropped_img = puzzle_img.crop(np.array(puzzle_img.size * 2) * np.array([0.05, 0.13, 0.95, 0.96]))
cropped_img.resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

In [ ]:
n = 10

In [ ]:
grid, timg = transcribe_grid(cropped_img, n, verbose=True)
grid

In [ ]:
timg.resize((n*32,n*32), Image.NEAREST)

### Apply Corrections

In [ ]:
# None

### Apply to Solver

In [ ]:
solution = complete_solve(grid)
solution

In [ ]:
draw_solution(cropped_img.copy(), solution).resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

## Example 5: u/Waitforsquirtle's from Reddit

Original post [here](https://www.reddit.com/r/puzzles/comments/1n4mw8r/star_battle_stuck_2_stars/)  
Reposted [here](https://www.reddit.com/r/puzzles/comments/1n4h2yr/stuck_on_2_star_star_battle/)

### Fetch the Data

In [ ]:
import requests
import io

In [ ]:
url = "https://preview.redd.it/star-battle-stuck-2-stars-v0-xhkn8xaxmamf1.jpeg?width=1080&crop=smart&auto=webp&s=3cc92cb2e3c67937857b9e5e69f4d439b6c7ff2a"
puzzle_img = Image.open(io.BytesIO(requests.get(url).content))
puzzle_img.resize((puzzle_img.size[0]//4, puzzle_img.size[1]//4))

### Apply the Transcription

In [ ]:
cropped_img = puzzle_img.crop(np.array(puzzle_img.size * 2) * np.array([0, 0, 1, 1]))
cropped_img.resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

In [ ]:
n = 9

In [ ]:
grid, timg = transcribe_grid(cropped_img, n, verbose=True)
grid

In [ ]:
timg.resize((n*32,n*32), Image.NEAREST)

### Apply Corrections

In [ ]:
# None

### Apply to Solver

In [ ]:
solution = complete_solve(grid, l = 2)
solution

In [ ]:
draw_solution(cropped_img.copy(), solution).resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

## Example 6: u/Big_Fox_3996's from Reddit

Original post [here](https://www.reddit.com/r/puzzles/comments/1nbriv1/queens_master_queendoku_game_strategy_help/)

### Fetch the Data

In [ ]:
import requests
import io

In [ ]:
url = "https://preview.redd.it/queens-master-queendoku-game-strategy-help-v0-xk25zt2nqynf1.jpeg?width=1080&crop=smart&auto=webp&s=bb0658281f4b8c53516b2a7e8bf28474441b411f"
puzzle_img = Image.open(io.BytesIO(requests.get(url).content))
puzzle_img.resize((puzzle_img.size[0]//4, puzzle_img.size[1]//4))

### Apply the Transcription

In [ ]:
cropped_img = puzzle_img.crop(np.array(puzzle_img.size * 2) * np.array([0.04, 0.05, 0.96, 0.94]))
cropped_img.resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

In [ ]:
n = 8

In [ ]:
grid, timg = transcribe_grid(cropped_img, n, verbose=True)
grid

In [ ]:
timg.resize((n*32,n*32), Image.NEAREST)

### Apply Corrections

In [ ]:
# None

### Apply to Solver

In [ ]:
solution = complete_solve(grid)
solution

In [ ]:
draw_solution(cropped_img.copy(), solution).resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

## Example 7: u/billuaa's from Reddit

Original post [here](https://www.reddit.com/r/puzzles/comments/1luluyi/linkedin_queens_dev_are_you_okay/)

### Fetch the Data

In [ ]:
import requests
import io

In [ ]:
url = "https://preview.redd.it/linkedin-queens-dev-are-you-okay-v0-pznrul1zvmbf1.jpeg?width=1080&crop=smart&auto=webp&s=480514cf29d5934d2ad3f7cfda8928a5499f0d66"
puzzle_img = Image.open(io.BytesIO(requests.get(url).content))
puzzle_img.resize((puzzle_img.size[0]//4, puzzle_img.size[1]//4))

### Apply the Transcription

In [ ]:
cropped_img = puzzle_img.crop(np.array(puzzle_img.size * 2) * np.array([0.04, 0.19, 0.96, 0.61]))
cropped_img.resize((cropped_img.size[0]//4, cropped_img.size[1]//4))

In [ ]:
n = 9

In [ ]:
grid, timg = transcribe_grid(cropped_img, n, verbose=True)
grid

In [ ]:
timg.resize((n*32,n*32), Image.NEAREST)

### Apply Corrections

In [ ]:
# None

### Apply to Solver

In [ ]:
solution = complete_solve(grid)
solution

In [ ]:
draw_solution(cropped_img.copy(), solution).resize((cropped_img.size[0]//4, cropped_img.size[1]//4))